In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, Flatten, Dense, Dropout,
    TimeDistributed, BatchNormalization, LSTM
)
from tensorflow.keras.callbacks import EarlyStopping

import numpy as np
from sklearn.model_selection import train_test_split

import optuna


2025-06-21 15:44:02.258592: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1750520642.442513      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1750520642.495912      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


### CNN Auto Hyperparameter Tuning

In [ ]:

# Load data dari file .npz
data_train_cnn = np.load('../data/processed/cnn_train_data.npz')

X_train_sub = data_train_cnn['data']
y_train_sub = data_train_cnn['label']

data_val_cnn = np.load('../data/processed/cnn_val_data.npz')

X_val_sub = data_val_cnn['data']
y_val_sub = data_val_cnn['label']

In [7]:

# Cek distribusi kelas pada data training
print("\nDistribusi kelas pada data training:")
classes_train, counts_train = np.unique(y_train_sub, return_counts=True)
for c, n in zip(classes_train, counts_train):
    print(f"  Kelas {int(c)}: {n} sampel")

# Cek distribusi kelas pada data validasi
print("\nDistribusi kelas pada data validasi:")
classes_val, counts_val = np.unique(y_val_sub, return_counts=True)
for c, n in zip(classes_val, counts_val):
    print(f"  Kelas {int(c)}: {n} sampel")



Distribusi kelas pada data training:
  Kelas 0: 2580 sampel
  Kelas 1: 5160 sampel

Distribusi kelas pada data validasi:
  Kelas 0: 2548 sampel
  Kelas 1: 5094 sampel


In [ ]:
# === Objective Function ===
def objective(trial):

    # Hyperparameters to tune
    learning_rate = trial.suggest_categorical("learning_rate", [1e-5, 1e-4, 1e-3, 1e-2, 1e-1])
    dropout_rate = trial.suggest_categorical("dropout_rate", [0.1, 0.2, 0.3, 0.4, 0.5])
    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128])


    model = Sequential([
        Conv2D(8,(3, 3), activation='relu', input_shape=(128, 128, 1), name='conv2d_1'),
        MaxPooling2D(2, 2, name='maxpool_1'),
        Conv2D(16, (3, 3), activation='relu', name='conv2d_2'),
        MaxPooling2D(2, 2, name='maxpool_2'),
        Flatten(name='flatten'),
        Dense(32, activation='relu', name='dense_1'),
        Dropout(dropout_rate, name='dropout_1'),
        Dense(1, activation='sigmoid', name='output')
    ])


    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    # Early stopping
    early_stop = EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True,
        verbose=1
    )

    # Fit model
    history = model.fit(
        X_train_sub, y_train_sub,
        epochs=20,
        batch_size=batch_size,
        validation_data=(X_val_sub, y_val_sub),
        callbacks=[early_stop],
        verbose=1
    )

    # Return validation accuracy
    val_acc = max(history.history["val_accuracy"])
    return val_acc



In [9]:
# === Run Optuna Study ===
study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective, n_trials=30)

# === Output best result ===
print("Best trial:")
trial = study.best_trial
print(f"  Accuracy: {trial.value}")
print("  Hyperparameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-06-21 15:47:13,167] A new study created in memory with name: no-name-7a35017e-2a3f-45a9-8ef2-320c0d9fe5f3


Epoch 1/20
121/121 ━━━━━━━━━━━━━━━━━━━━ 8s 37ms/step - accuracy: 0.6809 - loss: 0.5606 - val_accuracy: 0.7484 - val_loss: 0.4540
Epoch 2/20
121/121 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.7779 - loss: 0.4385 - val_accuracy: 0.7577 - val_loss: 0.4071
Epoch 3/20
121/121 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.8222 - loss: 0.3803 - val_accuracy: 0.8540 - val_loss: 0.3401
Epoch 4/20
121/121 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.8499 - loss: 0.3337 - val_accuracy: 0.8613 - val_loss: 0.3113
Epoch 5/20
121/121 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.8712 - loss: 0.2956 - val_accuracy: 0.8503 - val_loss: 0.3017
Epoch 6/20
121/121 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.8799 - loss: 0.2801 - val_accuracy: 0.8622 - val_loss: 0.2875
Epoch 7/20
121/121 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.8900 - loss: 0.2567 - val_accuracy: 0.8991 - val_loss: 0.2503
Epoch 8/20
121/121 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9023 - loss: 0.2367 - val_accu

[I 2025-06-21 15:47:46,456] Trial 0 finished with value: 0.9120649099349976 and parameters: {'learning_rate': 0.0001, 'dropout_rate': 0.3, 'batch_size': 64}. Best is trial 0 with value: 0.9120649099349976.


Epoch 1/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 8s 73ms/step - accuracy: 0.5878 - loss: 31.2170 - val_accuracy: 0.6666 - val_loss: 0.6366
Epoch 2/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.6691 - loss: 0.6353 - val_accuracy: 0.6666 - val_loss: 0.6366
Epoch 3/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.6583 - loss: 0.6425 - val_accuracy: 0.6666 - val_loss: 0.6366
Epoch 4/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.6623 - loss: 0.6400 - val_accuracy: 0.6666 - val_loss: 0.6368
Epoch 5/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.6625 - loss: 0.6393 - val_accuracy: 0.6666 - val_loss: 0.6368
Epoch 6/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.6796 - loss: 0.6281 - val_accuracy: 0.6666 - val_loss: 0.6366
Epoch 7/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.6726 - loss: 0.6339 - val_accuracy: 0.6666 - val_loss: 0.6370
Epoch 8/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.6639 - loss: 0.6395 - val_accuracy: 0.6666 - 

[I 2025-06-21 15:48:07,294] Trial 1 finished with value: 0.6665794253349304 and parameters: {'learning_rate': 0.1, 'dropout_rate': 0.3, 'batch_size': 128}. Best is trial 0 with value: 0.9120649099349976.


...

Epoch 1/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 7s 67ms/step - accuracy: 0.6600 - loss: 0.5536 - val_accuracy: 0.7768 - val_loss: 0.4069
Epoch 2/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.8038 - loss: 0.3831 - val_accuracy: 0.8377 - val_loss: 0.3165
Epoch 3/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 1s 19ms/step - accuracy: 0.8552 - loss: 0.3045 - val_accuracy: 0.9170 - val_loss: 0.2268
Epoch 4/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.8942 - loss: 0.2481 - val_accuracy: 0.9289 - val_loss: 0.2056
Epoch 5/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.9074 - loss: 0.2201 - val_accuracy: 0.9163 - val_loss: 0.2234
Epoch 6/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.9296 - loss: 0.1894 - val_accuracy: 0.9352 - val_loss: 0.1869
Epoch 7/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.9447 - loss: 0.1693 - val_accuracy: 0.9296 - val_loss: 0.1875
Epoch 8/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.9522 - loss: 0.1482 - val_accuracy: 0.9295 - v

[I 2025-06-21 15:58:50,755] Trial 28 finished with value: 0.9352263808250427 and parameters: {'learning_rate': 0.001, 'dropout_rate': 0.3, 'batch_size': 128}. Best is trial 15 with value: 0.9459565281867981.


Epoch 1/20
121/121 ━━━━━━━━━━━━━━━━━━━━ 7s 34ms/step - accuracy: 0.6789 - loss: 0.5776 - val_accuracy: 0.7705 - val_loss: 0.4488
Epoch 2/20
121/121 ━━━━━━━━━━━━━━━━━━━━ 2s 12ms/step - accuracy: 0.7811 - loss: 0.4456 - val_accuracy: 0.8300 - val_loss: 0.4093
Epoch 3/20
121/121 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.8288 - loss: 0.3792 - val_accuracy: 0.8648 - val_loss: 0.3243
Epoch 4/20
121/121 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.8683 - loss: 0.3165 - val_accuracy: 0.8746 - val_loss: 0.2860
Epoch 5/20
121/121 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.8826 - loss: 0.2834 - val_accuracy: 0.8448 - val_loss: 0.3069
Epoch 6/20
121/121 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9002 - loss: 0.2481 - val_accuracy: 0.9021 - val_loss: 0.2355
Epoch 7/20
121/121 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9075 - loss: 0.2319 - val_accuracy: 0.8881 - val_loss: 0.2512
Epoch 8/20
121/121 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9272 - loss: 0.2080 - val_accu

[I 2025-06-21 15:59:23,656] Trial 29 finished with value: 0.922664225101471 and parameters: {'learning_rate': 0.0001, 'dropout_rate': 0.3, 'batch_size': 64}. Best is trial 15 with value: 0.9459565281867981.


Best trial:
  Accuracy: 0.9459565281867981
  Hyperparameters:
    learning_rate: 0.001
    dropout_rate: 0.5
    batch_size: 32


### CNN-LSTM Auto Hyperparameter Tuning

In [ ]:
# Load data dari file .npz
data_train_cnn_lstm = np.load('../data/processed/cnn_lstm_train_data.npz')

data_val_cnn_lstm = np.load('../data/processed/cnn_lstm_val_data.npz')


X_train = data_train_cnn_lstm['data']
y_train = data_train_cnn_lstm['label']

X_val = data_val_cnn_lstm['data']
y_val = data_val_cnn_lstm['label']



In [12]:
# Lihat distribusi kelas pada data training
classes_train, counts_train = np.unique(y_train, return_counts=True)
print("Distribusi kelas pada data TRAIN:")
for cls, count in zip(classes_train, counts_train):
    print(f"  Kelas {cls}: {count} sampel")

# Lihat distribusi kelas pada data validasi
classes_val, counts_val = np.unique(y_val, return_counts=True)
print("\nDistribusi kelas pada data VALIDASI:")
for cls, count in zip(classes_val, counts_val):
    print(f"  Kelas {cls}: {count} sampel")


Distribusi kelas pada data TRAIN:
  Kelas 0: 2580 sampel
  Kelas 1: 5160 sampel

Distribusi kelas pada data VALIDASI:
  Kelas 0: 2548 sampel
  Kelas 1: 5094 sampel


In [ ]:

# === Objective Function ===
def objective(trial):
    # Hyperparameters to tune
    learning_rate = trial.suggest_categorical("learning_rate", [1e-5, 1e-4, 1e-3, 1e-2, 1e-1])
    dropout_rate = trial.suggest_categorical("dropout_rate", [0.1, 0.2, 0.3, 0.4, 0.5])
    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128])

    # Define model

    model = Sequential([
        TimeDistributed(Conv2D(8, (3, 3), activation='relu', padding='same'),
                        input_shape=(32, 128, 4, 1), name="conv2d_1"),
        TimeDistributed(BatchNormalization(), name="batchnorm_1"),
        TimeDistributed(MaxPooling2D((2, 2), padding="same"), name="maxpool_1"),

        TimeDistributed(Conv2D(16, (3, 3), activation='relu', padding='same'), name="conv2d_2"),
        TimeDistributed(MaxPooling2D((2, 2), padding="same"), name="maxpool_2"),

        TimeDistributed(Flatten(), name="flatten"),

        LSTM(64, return_sequences=True, name="lstm_1"),
        LSTM(32, return_sequences=False, name="lstm_2"),

        Dense(32, activation='relu', name="dense_1"),
        Dropout(dropout_rate, name="dropout_1"),

        Dense(1, activation='sigmoid', name="output")
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    # Early stopping
    early_stop = EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True,
        verbose=1
    )

    # Train model
    history = model.fit(
        X_train, y_train,
        epochs=20,
        batch_size=batch_size,
        validation_data=(X_val, y_val),
        callbacks=[early_stop],
        verbose=1
    )

    # Return best validation accuracy as objective
    val_acc = max(history.history['val_accuracy'])
    return val_acc



In [14]:
# === Run Optuna Study ===
study = optuna.create_study(direction="maximize", sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective, n_trials=30)

# === Output best result ===
print("Best trial:")
trial = study.best_trial
print(f"  Accuracy: {trial.value}")
print("  Hyperparameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")


[I 2025-06-21 16:03:05,944] A new study created in memory with name: no-name-6557810f-edd2-499f-b42e-f7391fc55759
/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/wrapper.py:27: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20
121/121 ━━━━━━━━━━━━━━━━━━━━ 35s 100ms/step - accuracy: 0.6632 - loss: 0.6318 - val_accuracy: 0.8328 - val_loss: 0.4805
Epoch 2/20
121/121 ━━━━━━━━━━━━━━━━━━━━ 7s 58ms/step - accuracy: 0.8363 - loss: 0.3797 - val_accuracy: 0.8213 - val_loss: 0.3636
Epoch 3/20
121/121 ━━━━━━━━━━━━━━━━━━━━ 7s 59ms/step - accuracy: 0.9179 - loss: 0.2230 - val_accuracy: 0.8300 - val_loss: 0.4623
Epoch 4/20
121/121 ━━━━━━━━━━━━━━━━━━━━ 7s 59ms/step - accuracy: 0.9478 - loss: 0.1570 - val_accuracy: 0.8190 - val_loss: 0.5051
Epoch 5/20
121/121 ━━━━━━━━━━━━━━━━━━━━ 7s 59ms/step - accuracy: 0.9711 - loss: 0.1036 - val_accuracy: 0.9486 - val_loss: 0.1393
Epoch 6/20
121/121 ━━━━━━━━━━━━━━━━━━━━ 7s 59ms/step - accuracy: 0.9839 - loss: 0.0668 - val_accuracy: 0.9163 - val_loss: 0.2507
Epoch 7/20
121/121 ━━━━━━━━━━━━━━━━━━━━ 7s 58ms/step - accuracy: 0.9782 - loss: 0.0681 - val_accuracy: 0.9687 - val_loss: 0.0878
Epoch 8/20
121/121 ━━━━━━━━━━━━━━━━━━━━ 7s 59ms/step - accuracy: 0.9904 - loss: 0.0428 - val_ac

[I 2025-06-21 16:05:57,853] Trial 0 finished with value: 0.9929338097572327 and parameters: {'learning_rate': 0.0001, 'dropout_rate': 0.3, 'batch_size': 64}. Best is trial 0 with value: 0.9929338097572327.


Epoch 1/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 31s 169ms/step - accuracy: 0.6426 - loss: 0.7538 - val_accuracy: 0.6666 - val_loss: 0.6372
Epoch 2/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 5s 86ms/step - accuracy: 0.6601 - loss: 0.6417 - val_accuracy: 0.6666 - val_loss: 0.6367
Epoch 3/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 5s 88ms/step - accuracy: 0.6548 - loss: 0.6459 - val_accuracy: 0.6666 - val_loss: 0.6371
Epoch 4/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 5s 87ms/step - accuracy: 0.6606 - loss: 0.6420 - val_accuracy: 0.6666 - val_loss: 0.6366
Epoch 5/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 5s 86ms/step - accuracy: 0.6726 - loss: 0.6330 - val_accuracy: 0.6666 - val_loss: 0.6366
Epoch 6/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 5s 87ms/step - accuracy: 0.6574 - loss: 0.6433 - val_accuracy: 0.6666 - val_loss: 0.6366
Epoch 7/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 5s 85ms/step - accuracy: 0.6680 - loss: 0.6359 - val_accuracy: 0.6666 - val_loss: 0.6374
Epoch 8/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 5s 85ms/step - accuracy: 0.6695 - loss: 0.6368 - val_accuracy: 0.6666 -

[I 2025-06-21 16:07:12,602] Trial 1 finished with value: 0.6665794253349304 and parameters: {'learning_rate': 0.1, 'dropout_rate': 0.3, 'batch_size': 128}. Best is trial 0 with value: 0.9929338097572327.


...

Epoch 1/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 30s 200ms/step - accuracy: 0.6590 - loss: 0.6188 - val_accuracy: 0.6833 - val_loss: 0.5105
Epoch 2/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 5s 85ms/step - accuracy: 0.7899 - loss: 0.4272 - val_accuracy: 0.7253 - val_loss: 0.4706
Epoch 3/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 5s 84ms/step - accuracy: 0.8606 - loss: 0.2969 - val_accuracy: 0.8108 - val_loss: 0.3939
Epoch 4/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 5s 85ms/step - accuracy: 0.9313 - loss: 0.1749 - val_accuracy: 0.8626 - val_loss: 0.3355
Epoch 5/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 5s 85ms/step - accuracy: 0.9714 - loss: 0.0888 - val_accuracy: 0.8274 - val_loss: 0.5361
Epoch 6/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 5s 85ms/step - accuracy: 0.9840 - loss: 0.0496 - val_accuracy: 0.7944 - val_loss: 0.9638
Epoch 7/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 5s 86ms/step - accuracy: 0.9914 - loss: 0.0285 - val_accuracy: 0.9093 - val_loss: 0.3475
Epoch 8/20
61/61 ━━━━━━━━━━━━━━━━━━━━ 5s 85ms/step - accuracy: 0.9836 - loss: 0.0506 - val_accuracy: 0.8871 -

[I 2025-06-21 17:20:46,964] Trial 28 finished with value: 0.9093169569969177 and parameters: {'learning_rate': 0.001, 'dropout_rate': 0.1, 'batch_size': 128}. Best is trial 6 with value: 0.9994765520095825.


Epoch 1/20
121/121 ━━━━━━━━━━━━━━━━━━━━ 32s 93ms/step - accuracy: 0.6505 - loss: 0.6423 - val_accuracy: 0.7649 - val_loss: 0.5325
Epoch 2/20
121/121 ━━━━━━━━━━━━━━━━━━━━ 7s 59ms/step - accuracy: 0.8368 - loss: 0.3957 - val_accuracy: 0.8818 - val_loss: 0.3123
Epoch 3/20
121/121 ━━━━━━━━━━━━━━━━━━━━ 7s 60ms/step - accuracy: 0.9433 - loss: 0.1845 - val_accuracy: 0.9706 - val_loss: 0.0995
Epoch 4/20
121/121 ━━━━━━━━━━━━━━━━━━━━ 7s 60ms/step - accuracy: 0.9813 - loss: 0.0882 - val_accuracy: 0.9334 - val_loss: 0.1774
Epoch 5/20
121/121 ━━━━━━━━━━━━━━━━━━━━ 7s 59ms/step - accuracy: 0.9883 - loss: 0.0548 - val_accuracy: 0.9857 - val_loss: 0.0505
Epoch 6/20
121/121 ━━━━━━━━━━━━━━━━━━━━ 7s 59ms/step - accuracy: 0.9909 - loss: 0.0480 - val_accuracy: 0.9850 - val_loss: 0.0495
Epoch 7/20
121/121 ━━━━━━━━━━━━━━━━━━━━ 7s 59ms/step - accuracy: 0.9972 - loss: 0.0227 - val_accuracy: 0.9403 - val_loss: 0.1702
Epoch 8/20
121/121 ━━━━━━━━━━━━━━━━━━━━ 7s 60ms/step - accuracy: 0.9938 - loss: 0.0276 - val_acc

[I 2025-06-21 17:23:36,913] Trial 29 finished with value: 0.9922795295715332 and parameters: {'learning_rate': 0.0001, 'dropout_rate': 0.3, 'batch_size': 64}. Best is trial 6 with value: 0.9994765520095825.


Best trial:
  Accuracy: 0.9994765520095825
  Hyperparameters:
    learning_rate: 0.001
    dropout_rate: 0.4
    batch_size: 32
